In [2]:
import cv2 as cv

live_feed = cv.VideoCapture(0 + cv.CAP_DSHOW)

live_feed.set(cv.CAP_PROP_SETTINGS, 1)

# Try to set the highest resolution your camera supports
live_feed.set(cv.CAP_PROP_FRAME_WIDTH, 1920)
live_feed.set(cv.CAP_PROP_FRAME_HEIGHT, 1080)

while True:
    # successfully read or not, the frame
    isTrue, frame = live_feed.read()

    # Method 1: Scale by factor (e.g., 1.5x bigger)
    scale_factor = 0.8
    width = int(frame.shape[1] * scale_factor)
    height = int(frame.shape[0] * scale_factor)
    resized_frame = cv.resize(frame, (width, height))

    cv.imshow("Video", resized_frame)

    # Wait 20ms for keypress; use bitwise mask (0xFF) to extract only
    # ASCII bits, exit if 'd' pressed.
    if cv.waitKey(20) & 0xFF == ord("q"):
        break

live_feed.release()
cv.destroyAllWindows()

Camera Calibration

In [ ]:
import numpy as np
import cv2 as cv
import glob
import os

# TODO sort out magic numbers when implementing the class.


def calibrate(show_pics=True):
    # get the directory of the current file
    current_dir = os.path.dirname(__file__)

    # Go up two levels to reach the root
    root_dir = os.path.dirname(os.path.dirname(current_dir))

    # Get the directory of the calibration images
    calibration_dir = os.path.join(root_dir, "resources", "calibration_images")

    img_path_list = glob.glob(os.path.join(calibration_dir, "*.jpg"))

    # Initialise
    num_rows = 9
    num_cols = 6
    term_criteria = (
        cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER,
        30,
        0.001,
    )
    world_points_current = np.zeros((num_rows * num_cols, 3), np.float32)
    world_points_list = []
    img_points_list = []

    # find corners
    for current_img_path in img_path_list:
        img_bgr = cv.imread(current_img_path)
        img_gray = cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)
        corners_found, corners_original = cv.findChessboardCorners(
            img_gray, (num_rows, num_cols), None
        )

        if corners_found:
            world_points_list.append(world_points_current)
            corners_refined = cv.cornerSubPix(
                img_gray,
                corners_original,
                (
                    11,
                    11,
                ),
                (-1, -1),
                term_criteria,
            )
            img_points_list.append(corners_refined)

            if show_pics:
                cv.drawChessboardCorners(
                    img_bgr,
                    (num_rows, num_cols),
                    corners_refined,
                    corners_found,
                )
                cv.imshow("Chessboard", img_bgr)
                cv.waitKey(500)

    cv.destroyAllWindows()

    # Do the actual calibration to find the intrinsics and extrinsics
    reprojection_error, camera_matrix, dist_coeff, r_vecs, t_vecs = (
        cv.calibrateCamera(
            world_points_list,
            img_points_list,
            img_gray.shape[::-1],
            None,
            None,
        )
    )
    print("Camera Matrix:\n", camera_matrix)
    print("Reprojection Error (pixels): {:.4f}".format(reprojection_error))

    # Save the calibration parameters in the current folder
    save_dir = os.path.join(current_dir, "camera_calibration.npz")
    np.savez(
        save_dir,
        reprojection_error=reprojection_error,
        camera_matrix=camera_matrix,
        dist_coeff=dist_coeff,
        r_vecs=r_vecs,
        t_vecs=t_vecs,
    )

    return camera_matrix, dist_coeff

    # https://youtu.be/H5qbRTikxI4?t=557 TODO then check with GPT

Remove Distortion

In [2]:
def remove_distortion(img, camera_matrix, dist_coeff):
    height, width = img.shape[:2]
    camera_matrix_new, roi = cv.getOptimalNewCameraMatrix(
        camera_matrix, dist_coeff, (width, height), 1, (width, height)
    )
    img_undistorted = cv.undistort(
        img, camera_matrix, dist_coeff, None, camera_matrix_new
    )

    return img_undistorted

In [3]:
calibrate()

NameError: name '__file__' is not defined